In this first part of the exercise we will build a convolutional neural network from scratch in Numpy.
We will also implement vanilla gradient descent and use it to train a classification head for the MNIST dataset.
MNIST is a dataset that contains images of the digits 0-9 in 28px resolution. We will use it to classify the digit based on the image.

Some remarks:
- make sure to carefully read the markdowns as well as the comments present in the code, they contain all the information that you will need to finish this exercise
- please add comments to the code you write, they do not have to be thorough but should provide a clear outline to what you are doing
- sometimes you merely have to complete the lines of the code, sometimes you need to write entire code blocks yourself
- There is a second part to this exercise (exercise_1.2)

First we will load the necessary packages.
Numpy for mathematical operations and torchvision as it contains the MNIST data that we will work with.

In [1]:
import numpy as np
from torchvision.datasets import MNIST

First we will load the training dataset and have a look at the first sample.
Note that the images are stored as PIL images and we will need to convert them to numpy arrays later.
The __getitem__() method loads the sample with the inserted index. The first entry of the sample is the image, the second is the label / corresponding digit.

In [2]:
train_data = MNIST(root="./", train=True, transform=None, target_transform=None, download=True)
img = train_data.__getitem__(0)[0]
display(img)

Lets have a look of some more samples.

In [3]:
for i in range(1,10):
    img = train_data.__getitem__(i)[0]
    label = train_data.__getitem__(i)[1]
    display(img)
    print(label)

0


4


1


9


2


1


3


1


4


The following is an example of how we can transform the data into numpy arrays.

In [4]:
img = np.array(train_data.__getitem__(0)[0])
label = np.array(train_data.__getitem__(0)[1])

28


Your first task is to write a convolutional layer in numpy. For simplicity, you only need to implement the layer such that it has only one input and output channel. Check slides 49-55 of the lecture for information on conv layers.

Processing input through the neural network to get some form of output is called a "forward" operation, make sure that the forward operation applies the convolution operation to an input of arbitrary size. The input to the forward function is supposed to be a "batched" numpy array, meaning that the layer can process multiple different inputs at once. Input and output arrays then consist of a batch-dimension; its common practice to use the first dimension as batch-dimension. Even input arrays that only contain one sample should consist of a batch-dimension (this can f.i. be done with np.expand_dims). Moreover, code your layer such that you can provide a list that contains the weights of the convolutional kernel, if no list is provided the kernel should be initialized with random weights.

Refer to slide 101 for batching.

In [7]:
class conv_layer():
    # implement init here
    # layer should operate with arbitrary (but reasonable) parameters
    def __init__(self, kernel_size: int, zero_padding_size: int, stride: int, weights: list = None):
        self.kernel_size = kernel_size
        self.zero_padding_size = zero_padding_size
        self.stride = stride
        self.weights = weights
        if self.weights is None:
            self.weights = np.random.rand(self.kernel_size, self.kernel_size) # init random kernel

    def add_zero_padding(self, image):
        height = image.shape[0]
        width = image.shape[1]
        new_height = height + 2 * self.zero_padding_size
        new_width = width + 2 * self.zero_padding_size
        padded_image = np.zeros((new_height, new_width))
        padded_image[self.zero_padding_size:self.zero_padding_size+height,
        self.zero_padding_size:self.zero_padding_size+width] = image
        return padded_image

    # implement conv operation here
    def forward(self, input):
        batch_size = input.shape[0]
        outputs = []
        for i in range(batch_size):
            input_sample = self.add_zero_padding(input[i])
            for x in range(1, input_sample.shape[1] - 1, self.stride):
                 for y in range(1, input_sample.shape[0] - 1, self.stride):
                     v = np.sum(input_sample[(y - self.kernel_size // 2):(y + (self.kernel_size // 2 + 1)),(x - self.kernel_size // 2):(x + (self.kernel_size // 2 + 1))] * self.weights)
                     input_sample[y, x] = v
            outputs.append(input_sample)
        output_batch = np.array(outputs)
        return output_batch

In [9]:
# Test your conv layer here
conv = conv_layer(3, 2, 1, None)
out = conv.forward(np.expand_dims(img, axis=0))
print(out)

[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.67262672e+12
   7.94052426e+12 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 7.36210307e+12
   3.47800896e+13 0.00000000e+00]
  ...
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.51904733e+20
   8.30980429e+20 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.58270959e+20
   8.68566432e+20 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]]]


Your next task is to write a fully connected layer. For details refer to slides 38-39,47 in the course material. 
Again, make sure that your layer can process a batched input and that weights can be provided manually.

In [107]:
class fc_layer():
    # implement init here
    def __init__(self, in_neurons: int, out_neurons: int, weights: list = None):
        ...
    # implement forward pass here (hint: its just a matrix multiplication)
    def forward(self, input):
        ...

Now we want to build or small convolutional neural network. 
Here are some activations that we use in the net. The softmax function transforms the output of our network into probabilites for each possible output (in our case the digits from 0-9).
Refer to slide 66 for more details.

In [108]:
def relu(x):
    return x * (x > 0)

def softmax(x):
    exp_vals = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_vals / np.sum(exp_vals, axis=-1, keepdims=True)

Your next task is to build a simple cnn with one conv layer followed by a fully connected layer.
The cnn shall be build such that input images with 28px resolution are mapped to an output dimension of 10 (digits 0-9, one output probability for each class).
Make sure that the conv layer is followed by relu activation before being fed into the fully connected layer. Also ensure that the dimensions are adjusted appropriately such that the output of the conv layer can be processed by the fc layer. Finally, the output shall be passed through the softmax function.

In [109]:
class simple_cnn():
    def __init__(self):
        ...
    def forward(self, input):
        ...

In [110]:
# test your cnn with an input of size (batch_size, 28, 28) here, output dim should be (batch_size, 10)
# Set the non-weight params of the conv layer to 3, 1 and 1

As the labels shown earlier contained the actual displayed digit, we want to transform them into a so called one-hot-encoded vector that contains a 1 at the position of the true class and otherwise 0s. Check slides 89 or https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html for reference.

In [111]:
def label_one_hot_encoding(x, dim=10):
    output = np.zeros(dim)
    output[x] = 1
    return output

The following function return the mean squared error of a target/label vector and a prediction. See slide 87 for reference.

In [112]:
def mse(target, pred):
    return np.mean(np.sum((target-pred)**2))

The final task of this exercise is to train a fully connected layer to predict the digit based on the corresponding 28px input.
To do so, you need to implement the backpropagation algorithm with respect to the above mse function.
Check slides 96-98 for references to the backpropagation algorithm.

Some remarks:
- you can ignore the conv layer and cnn from now on as we will only train a fully connected layer for simplicity
- the following backward function needs to compute the gradient of the passed weights of a fully connected layer for each weight and should have the same shape as the weights
- gradients over batches need to be averaged
- all of input, target, pred and fc_weights need to be used in order to compute the correct gradient

In [113]:
def backward_mse_to_fc(input, target, pred, fc_weights):
    gradients = np.zeros(fc_weights.shape)
    # compute gradients here


    return gradients

Next we need to implement a training loop. We want to feed each sample of the train data exactly once to our fc layer (one epoch). At the beginning of each training step we want to obtain the batched samples and one-hot-encoded labels. Then we feed the samples to our layer and obtain the predictions, which then need to be fed to the softmax function. Finally, we need to compute the gradients and perform basic gradient descent on the weights of the fc layer. Write the code such that arbitrary learning rates and batch sizes can be used.

See slide 99 for references on the gradient descent algorithm.

In [114]:
# define fc layer here
in_neurons = ...
out_neurons = ...
fc = fc_layer(in_neurons, out_neurons)

# define learning rate and batch_size
lr = ...
batch_size = ...
# num_samples = 60000
num_samples = len(train_data)

for i in range(num_samples // batch_size):
    # fill input and target batches here
    # make sure that labels are one_hot_encoded
    # make sure that image dimensions are changed such that the fc layer can handle it
    # the layer can process the input easier if you divide it by 255
    input = np.zeros((batch_size, in_neurons))
    targets = np.zeros((batch_size, out_neurons))

    # perform prediction here
    pred = ...

    # compute loss here, if you want also monitor your loss here to see if it improves during training
    loss = ...

    # compute gradients here
    gradients = backward_mse_to_fc(input, targets, pred, fc.weights)

    # update layer weights here
    fc.weights = ...

After we ran the training loop, we want to test our model on hold-out test data.
First we load the test data. (10000 samples)

In [ ]:
test_data = MNIST(root="./", train=False, transform=None, target_transform=None, download=True)


As the final task, compute the accuracy of your model on the test data.
For simplicity, you can use a batch_size of 1 here if you want to.

Check slides 103-104 for references to the accuracy score.

In [ ]:
num_test_samples = len(test_data)
results = []

for i in range(num_test_samples):
    # load sample and label here
    # dont forget: change of dims for forward pass through fc, divide by 255 if you did so before, one-hot-encoding, adding batch_dimension
    input = ...
    label = ...

    # compute prediction
    pred = ...

    # use prediction to update the accuracy score or store in results lists 

# compute and report final accuracy



Retrain the fc layer at least 2 more times with different batch_sizes and or learning_rates. 
Report the results either in this submission notebook or just report them in text.